<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Обзор-данных" data-toc-modified-id="Обзор-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Обзор данных</a></span></li><li><span><a href="#Исследование-баланса-классов" data-toc-modified-id="Исследование-баланса-классов-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование баланса классов</a></span></li><li><span><a href="#Обучение-моделей-без-учета-дисбаланса-классов" data-toc-modified-id="Обучение-моделей-без-учета-дисбаланса-классов-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Обучение моделей без учета дисбаланса классов</a></span></li><li><span><a href="#Обучение-моделей-с-учетом-дисбаланса-классов-(взвешиванием-классов)" data-toc-modified-id="Обучение-моделей-с-учетом-дисбаланса-классов-(взвешиванием-классов)-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Обучение моделей с учетом дисбаланса классов (взвешиванием классов)</a></span></li><li><span><a href="#Обучение-моделей-с-учетом-дисбаланса-классов-(апсемплинг)" data-toc-modified-id="Обучение-моделей-с-учетом-дисбаланса-классов-(апсемплинг)-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Обучение моделей с учетом дисбаланса классов (апсемплинг)</a></span></li><li><span><a href="#Финальное-тестирование" data-toc-modified-id="Финальное-тестирование-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Финальное тестирование</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Общий вывод</a></span></li></ul></div>

# Проект для "Бета-банка"

**Описание проекта**  
Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.  
Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Нам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком.  
Постройте модель с предельно большим значением F1-меры.  
Источник данных: https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling  


*Описание данных*  
Признаки  
RowNumber — индекс строки в данных  
CustomerId — уникальный идентификатор клиента  
Surname — фамилия  
CreditScore — кредитный рейтинг  
Geography — страна проживания  
Gender — пол  
Age — возраст  
Tenure — количество недвижимости у клиента  
Balance — баланс на счёте  
NumOfProducts — количество продуктов банка, используемых клиентом  
HasCrCard — наличие кредитной карты  
IsActiveMember — активность клиента  
EstimatedSalary — предполагаемая зарплата  

Целевой признак  
Exited — факт ухода клиента  

## Обзор данных

In [1]:
import pandas as pd
import random
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

In [2]:
import warnings
warnings.filterwarnings('ignore') 

In [3]:
#df = pd.read_csv(r'C:\Users\Susanna\Downloads\Churn.csv')
df = pd.read_csv(r'/datasets/Churn.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [347]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


**Вывод**    
Таблица содержит 10000 строк и 14 столбцов. Пропуски есть только в одном столбце Tenure. Пропусков 10%. Это слишком много чтобы просто их удалить.  
Посмотрим как Tenure коррелирует с другими столбцами.

In [348]:
corr=df.corr().sort_values(by='Tenure', ascending=False)
corr['Tenure']

Tenure             1.000000
HasCrCard          0.027232
NumOfProducts      0.011979
EstimatedSalary    0.010520
CreditScore       -0.000062
RowNumber         -0.007322
Balance           -0.007911
Age               -0.013134
Exited            -0.016761
CustomerId        -0.021418
IsActiveMember    -0.032178
Name: Tenure, dtype: float64

In [349]:
df['Tenure'].value_counts()

1.0     952
2.0     950
8.0     933
3.0     928
5.0     927
7.0     925
4.0     885
9.0     882
6.0     881
10.0    446
0.0     382
Name: Tenure, dtype: int64

**Вывод**    
Tenure практически никак не коррелирует с другими столбцами. Сложно заполнить пропуски в зависимости от содержания других столбцов. Считаю разумным заполнить пропуски случайным образом в пределах минимума и максимума всех возможных вариантов Tenure.

In [350]:
def random_tenure(row):
    if row['Tenure']!=row['Tenure']:
        return random.randint(df['Tenure'].min(), df['Tenure'].max())
    else:
        return row['Tenure']
df['Tenure']=df.apply(random_tenure, axis=1)

In [351]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


**Вывод**  
Таблица содержит 10000 строк и 14 столбцов. Все пропуски заполнены.

## Исследование баланса классов

In [352]:
#Фамилии нам для исследования не нужны
df2=df.drop(['Surname','RowNumber','CustomerId'], axis=1)


#Преобразую категориальные признаки в численные техникой прямого кодирования One-Hot Encoding
df2=pd.get_dummies(df2, drop_first=True)
df2.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,0,1,0


In [353]:
#Посмотрим насколько не сбалансирован датасет
df2['Exited'].value_counts()

0    7963
1    2037
Name: Exited, dtype: int64

**Вывод**  
Значений ноль в 4 раза больше единц.

## Обучение моделей без учета дисбаланса классов

In [354]:
#Разделю исходные данные на обучающую, валидационную и тестовую выборки.  
features=df2.drop(['Exited'], axis=1)
target=df2['Exited']

features_train, features_valid_test, target_train, target_valid_test = train_test_split(
    features, target, test_size=0.4, random_state=12345)

features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid_test, target_valid_test, test_size=0.5, random_state=12345)

In [355]:
features_train.shape, features_valid.shape, features_test.shape

((6000, 11), (2000, 11), (2000, 11))

In [356]:
#Создам функци,которые будут печатать метрики качества

def DecisionTree (df):
    
    predictions_valid=model.predict(features_valid)
    probabilities_valid = model.predict_proba(features_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    
    f1=f1_score(target_valid, predictions_valid)
    auc_roc=roc_auc_score(target_valid, probabilities_one_valid)
    
    df.loc[i, 'recall']=recall_score(target_valid, predictions_valid)
    df.loc[i, 'precision']=precision_score(target_valid, predictions_valid)
    df.loc[i, 'f1']=f1
    df.loc[i, 'auc_roc']=auc_roc

In [357]:
def RandomForest (df):
    
    predictions_valid=model.predict(features_valid)
    probabilities_valid = model.predict_proba(features_valid)
   
    f1=f1_score(target_valid, predictions_valid)
    auc_roc=roc_auc_score(target_valid, probabilities_one_valid)
    df.loc[k, 'recall']=recall_score(target_valid, predictions_valid)
    df.loc[k, 'precision']=precision_score(target_valid, predictions_valid)
    df.loc[k, 'max_depth']=i
    df.loc[k, 'n_estim']=j
    df.loc[k, 'f1']=f1
    df.loc[k, 'auc_roc']=auc_roc

In [358]:
def LogisticReg (target_valid, predictions_valid, probabilities_valid):
    f1=f1_score(target_valid, predictions_valid)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc=roc_auc_score(target_valid, probabilities_one_valid)
    recall=recall_score(target_valid, predictions_valid)
    precision=precision_score(target_valid, predictions_valid)
    
    print('recall:', recall)
    print('precision:', precision)
    print('f1:', f1)
    print('auc_roc:', auc_roc)
    print()

**DecisionTreeClassifier**

In [359]:
df11 = pd.DataFrame()

for i in range(1, 100):
    model = DecisionTreeClassifier(random_state=12345, max_depth=i)
    model.fit(features_train, target_train)
    DecisionTree(df11)

    
df11.sort_values(by='f1', ascending=False)


,recall,precision,f1,auc_roc
6,0.447368,0.782427,0.569254,0.817209
9,0.476077,0.676871,0.558989,0.785314
4,0.437799,0.750000,0.552870,0.813428
8,0.428230,0.752101,0.545732,0.807972
7,0.418660,0.760870,0.540123,0.817286
...,...,...,...,...
15,0.471292,0.517060,0.493116,0.677793
20,0.492823,0.488152,0.490476,0.679886
19,0.485646,0.489157,0.487395,0.679885
3,0.284689,0.826389,0.423488,0.797344


**RandomForestClassifier**

In [360]:
df=pd.DataFrame()
k=0
for i in range(1,11, 2):
    for j in range(1,51):
        k+=1
        model = RandomForestClassifier(random_state=12345, max_depth=i, n_estimators=j)
        model.fit(features_train, target_train)
        RandomForest(df12)
    
df12.sort_values(by='f1', ascending=False)

,recall,precision,max_depth,n_estim,f1,auc_roc
842,0.449761,0.843049,9.0,42.0,0.586583,0.6825
841,0.449761,0.839286,9.0,41.0,0.585670,0.6825
843,0.449761,0.835556,9.0,43.0,0.584759,0.6825
828,0.449761,0.835556,9.0,28.0,0.584759,0.6825
844,0.447368,0.842342,9.0,44.0,0.584375,0.6825
...,...,...,...,...,...,...
32,0.000000,0.000000,1.0,32.0,0.000000,0.6825
30,0.000000,0.000000,1.0,30.0,0.000000,0.6825
29,0.000000,0.000000,1.0,29.0,0.000000,0.6825
28,0.000000,0.000000,1.0,28.0,0.000000,0.6825


**LogisticRegression**

In [361]:
model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_train, target_train)
predictions_valid=model.predict(features_valid)
probabilities_valid = model.predict_proba(features_valid)

LogisticReg(target_valid, predictions_valid, probabilities_valid)

recall: 0.04784688995215311
precision: 0.3389830508474576
f1: 0.08385744234800838
auc_roc: 0.6728053036856018



**Вывод**    
Результаты не удовлетворительные.
Ни на одной моделе не удалось добиться f1 больше 0.59.  
Самые плохие результаты показала модель логистический регрессии.

## Обучение моделей с учетом дисбаланса классов (взвешиванием классов)

**DecisionTreeClassifier**

In [362]:
df21=pd.DataFrame()
for i in range(1, 200):
    model = DecisionTreeClassifier(random_state=12345, max_depth=i, class_weight='balanced')
    model.fit(features_train, target_train)
    
    DecisionTree(df21)

df21.sort_values(by='f1', ascending=False)

,recall,precision,f1,auc_roc
5,0.669856,0.536398,0.595745,0.831792
6,0.700957,0.459969,0.555450,0.796808
7,0.660287,0.466216,0.546535,0.786934
9,0.660287,0.461538,0.543307,0.767529
8,0.686603,0.447738,0.542021,0.780696
...,...,...,...,...
84,0.466507,0.492424,0.479115,0.669726
85,0.466507,0.492424,0.479115,0.669726
199,0.466507,0.492424,0.479115,0.669726
22,0.459330,0.492308,0.475248,0.666936


**RandomForestClassifier**

In [363]:
df22=pd.DataFrame()
k=0
for i in range(1,10, 2):
    for j in range(1,51):
        k+=1
        model = RandomForestClassifier(random_state=12345, max_depth=i, n_estimators=j, class_weight='balanced')
        model.fit(features_train, target_train)
        RandomForest(df22)
        
df22.sort_values(by='f1', ascending=False)

,recall,precision,max_depth,n_estim,f1,auc_roc
171,0.684211,0.584867,7.0,21.0,0.630650,0.6825
181,0.686603,0.578629,7.0,31.0,0.628009,0.6825
179,0.688995,0.576000,7.0,29.0,0.627451,0.6825
194,0.693780,0.570866,7.0,44.0,0.626350,0.6825
177,0.679426,0.579592,7.0,27.0,0.625551,0.6825
...,...,...,...,...,...,...
11,0.655502,0.314220,1.0,11.0,0.424806,0.6825
9,0.655502,0.314220,1.0,9.0,0.424806,0.6825
8,0.655502,0.314220,1.0,8.0,0.424806,0.6825
2,0.145933,0.835616,1.0,2.0,0.248473,0.6825


**LogisticRegression**

In [365]:
model = LogisticRegression(random_state=12345, solver='liblinear', class_weight='balanced')
model.fit(features_train, target_train)
predictions_valid=model.predict(features_valid)
probabilities_valid = model.predict_proba(features_valid)

LogisticReg(target_valid, predictions_valid, probabilities_valid)

recall: 0.6746411483253588
precision: 0.3401688781664656
f1: 0.4522854851643945
auc_roc: 0.7204268716844405



**Вывод**    
Результаты улучшились.  
Есть модели, которые показывают результат f1 больше 0.59.

## Обучение моделей с учетом дисбаланса классов (апсемплинг)

In [366]:
#Апсемплинг
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

features_upsampled, target_upsampled = upsample(features_train, target_train, 4)

**DecisionTreeClassifier**

In [367]:
df31=pd.DataFrame()
for i in range(1, 11):
    model = DecisionTreeClassifier(random_state=12345, max_depth=i)
    model.fit(features_upsampled, target_upsampled)
    DecisionTree(df31)
   

df31.sort_values(by='f1', ascending=False)

,recall,precision,f1,auc_roc
5,0.669856,0.536398,0.595745,0.831792
6,0.705742,0.461659,0.558184,0.801535
7,0.665072,0.469595,0.550495,0.791720
8,0.681818,0.455272,0.545977,0.779356
9,0.660287,0.462312,0.543842,0.767128
2,0.662679,0.457096,0.541016,0.750181
3,0.662679,0.457096,0.541016,0.798047
10,0.648325,0.447195,0.529297,0.762474
4,0.772727,0.400744,0.527778,0.819085
1,0.586124,0.435169,0.499490,0.692557


**RandomForestClassifier**

In [368]:
df32=pd.DataFrame()
k=0
for i in range(1,10,2):
    for j in range(1,51):
        k+=1
        model = RandomForestClassifier(random_state=12345, max_depth=i, n_estimators=j)
        model.fit(features_upsampled, target_upsampled)
        RandomForest(df32)
    
df32.sort_values(by='f1', ascending=False)

,recall,precision,max_depth,n_estim,f1,auc_roc
248,0.708134,0.561670,9.0,48.0,0.626455,0.6825
249,0.705742,0.560837,9.0,49.0,0.625000,0.6825
247,0.705742,0.559772,9.0,47.0,0.624339,0.6825
250,0.703349,0.561069,9.0,50.0,0.624204,0.6825
243,0.708134,0.557439,9.0,43.0,0.623815,0.6825
...,...,...,...,...,...,...
51,0.811005,0.342770,3.0,1.0,0.481876,0.6825
8,0.655502,0.314220,1.0,8.0,0.424806,0.6825
9,0.655502,0.314220,1.0,9.0,0.424806,0.6825
2,0.145933,0.835616,1.0,2.0,0.248473,0.6825


**LogisticRegression**

In [370]:
model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_upsampled, target_upsampled)
predictions_valid=model.predict(features_valid)
probabilities_valid = model.predict_proba(features_valid)

LogisticReg(target_valid, predictions_valid, probabilities_valid)

recall: 0.6746411483253588
precision: 0.34057971014492755
f1: 0.4526484751203852
auc_roc: 0.7203588214300837



**Вывод**   
С учетом дисбаланса многие модели стали показывать неплохие результаты.  
Наилуший результат F1=0.63 и auc_roc=0.69 у модели случайного леса с параметрами max_depth=8 и n_estimators=48.   
Буду использовать эту модель для проверки на тестовой выборке.

## Финальное тестирование

In [376]:
model = RandomForestClassifier(random_state=12345, n_estimators=21, max_depth=7, class_weight='balanced')
model.fit(features_train, target_train)

#Проверяю качество модели на тестовой выборке
predictions_test=model.predict(features_test)
probabilities_test = model.predict_proba(features_test)
   
f1=f1_score(target_test, predictions_test)
probabilities_one_test = probabilities_test[:, 1]
auc_roc=roc_auc_score(target_test, probabilities_one_test)
recall=recall_score(target_test, predictions_test)
precision=precision_score(target_test, predictions_test)
    
print('recall:', recall)
print('precision:', precision)
print('f1:', f1)
print('auc_roc:', auc_roc)
print()

recall: 0.6666666666666666
precision: 0.5391969407265774
f1: 0.5961945031712473
auc_roc: 0.8474855000442232



**Вывод**    
Модель хорошая, удовлетворяет заданным условиям. При проверке на тестовой выборке значения F1=0.60, auc_roc=0.85.

## Общий вывод  

В исходном датасете есть дисбаланс целевого признака. Нулей примерно в 4 раза больше, чем единиц.  
Я пробовала обучать модели решающего дерева, случайного леса и логистической ргрессии без учета дисбаланса. Результаты получились не очень хорошими.  
После учета дисбаланса результаты значительно улучшились.  
Наилучший результат получился после того как модель случайного леса была обучена на увеличенной обучающей выборке и показала результат F1=0.63 и auc_roc=0.68.  
При проверке на тестовой выборке модель показала результаты  F1=0.60, auc_roc=0.85.